In [4]:
from arcgis.gis import GIS
from arcgis.geocoding import geocode
from arcgis.raster.functions import *
from arcgis import geometry
import traceback
import pandas as pd
from IPython.display import HTML
from arcgis.network import ClosestFacilityLayer
from arcgis import geocoding
from arcgis.features import Feature, FeatureSet
from IPython.display import clear_output
from ipywidgets import  HBox

In [2]:
conda install jupyter_dashboards -c conda-forge -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


## Map

In [7]:
# connect as an anonymous user
gis = GIS()

# search for the map
our_map = gis.content.search("open street map", 'Imagery Layer', outside_org=True)[0]
my_map = our_map.layers[0]


#displaying the map
m = gis.map(location.value)
m.add_layer(my_map)
display(m)

MapView(layout=Layout(height='400px', width='100%'))

Bhoiwada Smashanbhumi Marg, Parel, Mumbai, Maharashtra, 400012


## Concern Area

In [6]:
import ipywidgets as widgets

# text box widget
location = widgets.Text(value='', placeholder='enter place',
                        description='Location:', disabled=False)

# command button widget
gobtn = widgets.Button(description='Go', disabled=False,
                       button_style='', tooltip='Go', icon='')



Updated_AllhospitalNameList = []
# define what happens whent the command button is clicked
def on_gobutton_clicked(b):
    
    # Connect to your GIS
    gis = GIS("home")

    try:
        analysis_url = gis.properties.helperServices.closestFacility.url
#         print("Closest Facility URL successfully retrieved")
    except Exception as e:
        traceback.print_exc()
        raise Exception(f"Network Analysis is not properly configured "\
                        f"on your Organization: {e}")


    cf_layer = ClosestFacilityLayer(analysis_url, gis = gis)
    
    # geocode the place name and set that as the map's extent
    area = geocode(location.value)[0]
    m.extent = area['extent']
    
    #for updating hospital list
    csv_file = '/arcgis/home/HospitalData.csv'
    AllDataHospital = pd.read_csv(csv_file,encoding='latin1')

    Thershold = 5
    query = location.value

    city  = query.split(',')[0].rstrip('\t').lstrip('\t')
    state = query.split(',')[1].rstrip('\t').lstrip('\t')

    AllhospitalNameList = []
    Filtered_Hospital_data = AllDataHospital.groupby('State_Name').get_group(state).groupby('District_Name').get_group(city)
    for index, entry in Filtered_Hospital_data.iterrows():
        name = str(entry['Health Facility Name'] ).strip('\n') + ','  
        address = str(entry['Address'] ).strip('\n') + ','  
        street = str(entry['street'] ).strip('\n')  + ','  
        locality = str(entry['locality'] ).strip('\n') + ','  
        pincode = str(entry['pincode'] ).strip('\n')+ ','  
        if locality == 'nan,': locality = " "
        hospitalAddress = name + address + street + locality + pincode
        AllhospitalNameList.append(hospitalAddress)
        
    
    Updated_AllhospitalNameList.append(AllhospitalNameList)
    
gobtn.on_click(on_gobutton_clicked)

location_items = [location, gobtn]
widgets.HBox(location_items)

In [8]:
# Connect to your GIS
gis = GIS("home")

try:
    analysis_url = gis.properties.helperServices.closestFacility.url
#     print("Closest Facility URL successfully retrieved")
except Exception as e:
    traceback.print_exc()
    raise Exception(f"Network Analysis is not properly configured "\
                    f"on your Organization: {e}")
    
    
cf_layer = ClosestFacilityLayer(analysis_url, gis = gis)

In [10]:
hospitals_addresses = Updated_AllhospitalNameList[-1]


hosp_feat_list = []

for address in hospitals_addresses:
    hit = geocoding.geocode(address)[0]
    hosp_feat = Feature(geometry=hit['location'],
                        attributes=hit['attributes'])
    hosp_feat_list.append(hosp_feat)
    
    
    
hospitals_fset = FeatureSet(features=hosp_feat_list, 
                            geometry_type='esriGeometryPoint', 
                            spatial_reference={'latestWkid': 4326})

In [11]:
#save coordinates of a point on clicking on map and drawing it on map as red point using the function auto_on_getlocbtn_clicked(x,y)
def find_addr(m, g):
    try:
        m.draw(g)
        geocoded = geocoding.reverse_geocode(g)
        auto_on_getlocbtn_clicked(geocoded['location']['x'], geocoded['location']['y'])
        print(geocoded['address']['Match_addr'])
    except:
        print("Couldn't match address. Try another place...")
m.on_click(find_addr)

## Hospital In Area

In [12]:
# create the action button
drawbtn = widgets.Button(description='Plot hospitals in area', disabled=False,
                         button_style='', tooltip='click to plot nearby hospitals',
                         icon='')

def on_drawbtn_clicked(b):
    #hospital database

#     hospitals_addresses = ['MPCT Hospital, C 7, Budhyadev Mandir Marg, Sector 4 , Mumbai - 400705', 
#                            'Sion - Trombay Road, Suman Nagar, Chembur East Chembur, Mumbai - 400071',
#                            'A - 791, Bandra Reclamation, Bandra West, Bandra, Mumbai - 400050',
#                            '15 G, Dr. Deshmukh Marg, Pedder Road, Opposite Sophia College Byculla, Mahalaxmi, Mumbai - 400026',
#                            'A- 15/17, Maharshi Karve Marg, Girgaon South Mumbai, Mumbai - 400004'
#                           ]

    hospitals_addresses = Updated_AllhospitalNameList[-1]



    hosp_feat_list = []

    for address in hospitals_addresses:
        hit = geocoding.geocode(address)[0]
        hosp_feat = Feature(geometry=hit['location'],
                            attributes=hit['attributes'])
        hosp_feat_list.append(hosp_feat)



    hospitals_fset = FeatureSet(features=hosp_feat_list, 
                                geometry_type='esriGeometryPoint', 
                                spatial_reference={'latestWkid': 4326})
    
    
    m.draw(hospitals_fset,symbol={"type": "esriSMS","style": "esriSMSDiamond","size": 12, "color":[76,115,0,255]})
    
drawbtn.on_click(on_drawbtn_clicked)
drawbtn

Button(description='Plot hospitals in area', style=ButtonStyle(), tooltip='click to plot nearby hospitals')

In [13]:
incident_fset1 = []
def auto_on_getlocbtn_clicked(x,y):
    
    
    # Connect to your GIS
    gis = GIS("home")

    try:
        analysis_url = gis.properties.helperServices.closestFacility.url
#         print("Closest Facility URL successfully retrieved")
    except Exception as e:
        traceback.print_exc()
        raise Exception(f"Network Analysis is not properly configured "\
                        f"on your Organization: {e}")


    cf_layer = ClosestFacilityLayer(analysis_url, gis = gis)
    
    
    
    
    reverse_geocode = geocoding.reverse_geocode(
    {"x": x, 
     "y": y})

    incident_feature = Feature(geometry=reverse_geocode['location'], 
                           attributes=reverse_geocode['address'])


    incident_fset = FeatureSet([incident_feature],
                           geometry_type='esriGeometryPoint',
                           spatial_reference={'latestWkid': 4326})
    
    
    m.draw(incident_fset,
          symbol={"type": "esriSMS","style": "esriSMSDiamond",
                  "size": 12, "color":[255,0,0,255]})
    incident_fset1.append(incident_fset)

## Optimal routes from incident area 

In [14]:
# create the action button
import random
drawbtn = widgets.Button(description='Plot nearest hospital routes', disabled=False,
                         button_style='', tooltip='click to plot route to nearby hospitals',
                         icon='')

#creating an empty list of routes to display as popup
routes = []


def auto_on_drawbtn_clicked(b):
    
    
    result = cf_layer.solve_closest_facility(
    incidents=incident_fset1[-1],
    facilities=hospitals_fset,
    default_target_facility_count=5,
    return_facilities=True,
    impedance_attribute_name='TravelTime',
    accumulate_attribute_names=['Kilometers','TravelTime'])

    result.keys()
    result['routes'].keys()
    result['routes']['features'][0].keys()

    line_feat_list = []
    for line_dict in result['routes']['features']:
        f1 = Feature(line_dict['geometry'], line_dict['attributes'])
        line_feat_list.append(f1)

    routes_fset = FeatureSet(line_feat_list, 
        geometry_type=result['routes']['geometryType'],
        spatial_reference= result['routes']['spatialReference'])
    
    ##df for popup
    
        
    routes.append(routes_fset)
    m.draw(routes_fset)
    
    
    
    
#     print(routes_fset)
# create the action button



drawbtn.on_click(auto_on_drawbtn_clicked)

drawbtn

Button(description='Plot nearest hospital routes', style=ButtonStyle(), tooltip='click to plot route to nearby…

In [18]:

def showoutput(routes):
    df1 = routes[-1].sdf.sort_values(by=['Name'])
    #     print(df1['FacilityID'])
    filteredaddress = [hospitals_addresses[i] for i in df1['FacilityID']]

    df1['Hospital_addresses'] = filteredaddress

    pd.set_option('max_colwidth', 80)

    df1[['Hospital_addresses','Total_Miles',
         'Total_TravelTime']].sort_values(by=['Total_TravelTime'])


    amb = pd.DataFrame({'ambulance_availability' : [random.randint(1,10) for _ in range(0,5)]})

    df1 = df1.join(amb)

    finaldf = df1[['Hospital_addresses','Total_Miles',
         'Total_TravelTime', 'ambulance_availability']].sort_values(by=['Total_TravelTime'])
    print(finaldf.to_markdown())
    
clrbtnshowresult = widgets.Button(description='Show result', disabled=False,
                         button_style='', tooltip='Clear',
                         icon='')
def on_clrbtn_clicked_output(b):
    showoutput(routes)

def on_clrbtn_clicked_clear_output(b):
    clear_output()
    
clrbtnshowresult.on_click(on_clrbtn_clicked_output)
clrbtnshowresult

Button(description='Show result', style=ButtonStyle(), tooltip='Clear')

|    | Hospital_addresses                                                                                                                                   |   Total_Miles |   Total_TravelTime |   ambulance_availability |
|---:|:-----------------------------------------------------------------------------------------------------------------------------------------------------|--------------:|-------------------:|-------------------------:|
|  0 | Antop Hill,Antop Hill Mun.Disp,Shaikh Mistry Marg,Antop Hill Mumbai,400037.0,                                                                        |      0.320032 |            2.01045 |                        7 |
|  1 | All India Institute of Physical Medicine and Rehabilitation (AIIPMR), Maharashtra,K. Khadye Marg,,Haji Ali,Mahalaxmi, Mumbai, Maharashtra, 400034.0, |      0.367449 |            2.61309 |                        3 |
|  2 | Kumbharwada,Valmiki Nagar  MATUMGA LABOUR CAMP,nan,Mumbai,400019.0,                                      

In [17]:
!pip install tabulate

In [16]:
# create the action button
clrbtnMap = widgets.Button(description='Clear Map', disabled=False,
                         button_style='', tooltip='Clear',
                         icon='')
clrbtnoutput = widgets.Button(description='Clear Output', disabled=False,
                         button_style='', tooltip='Clear',
                         icon='')

    
def on_clrbtn_clicked(b):
    m.clear_graphics()
    
clrbtnoutput.on_click(on_clrbtn_clicked_clear_output)


clrbtnMap.on_click(on_clrbtn_clicked)
HBox([clrbtnMap, clrbtnoutput])